In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.9MB/s 


In [3]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix two/dw_matrix_car"


/content/drive/My Drive/Colab Notebooks/matrix/matrix two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance (df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [7]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision tree

In [27]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015937)

## Random forest

In [28]:
model = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=50)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [16]:
xgb_params = {
    'max_depth' : 5,
    'n_eestimators' : 50,
    'seed' : 0,
    'learning_rate' : 0.1
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[18:13:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:13:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:14:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11862.434165529698, 264.2411021793361)

In [17]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp =PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[18:18:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1341 ± 0.0029,param_rok-produkcji__cat
0.1037 ± 0.0028,param_stan__cat
0.0956 ± 0.0029,param_napęd__cat
0.0501 ± 0.0020,param_skrzynia-biegów__cat
0.0442 ± 0.0014,param_moc__cat
0.0438 ± 0.0018,param_faktura-vat__cat
0.0401 ± 0.0008,param_marka-pojazdu__cat
0.0216 ± 0.0007,param_typ__cat
0.0182 ± 0.0012,feature_kamera-cofania__cat
0.0175 ± 0.0006,param_wersja__cat


In [19]:
feats = [
'param_rok-produkcji__cat',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

len(feats)

20

In [29]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[18:48:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:48:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:48:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10067.086619567794, 35.57616233370741)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))  

In [0]:
feats = [
'param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

In [37]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[18:54:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:54:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:54:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10067.086619567794, 35.57616233370741)

In [45]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(str(x).split(' ')[0]))
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x)) 

feats = [
'param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[19:01:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8698.488115726954, 52.63025457895438)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(x.split('cm')[0].replace(' ', '')))

In [43]:
feats = [
'param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[19:00:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:00:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:00:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8580.61650667904, 58.180956552556225)